In [1]:
import gym
import numpy as np
import torch
import matplotlib.pyplot as plt
import time

In [2]:
from gym.wrappers import Monitor

In [3]:
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

In [4]:
import math
import copy

In [6]:
class CartPoleAI(nn.Module):
        def __init__(self):
            super().__init__()
            self.fc = nn.Sequential(
                        nn.Linear(4,128, bias=True),
                        nn.ReLU(),
                        nn.Linear(128,2, bias=True),
                        nn.Softmax(dim=1)
                        )

                
        def forward(self, inputs):
            x = self.fc(inputs)
            return x

In [7]:
def init_weights(m):
    
        # nn.Conv2d weights are of shape [16, 1, 3, 3] i.e. # number of filters, 1, stride, stride
        # nn.Conv2d bias is of shape [16] i.e. # number of filters
        
        # nn.Linear weights are of shape [32, 24336] i.e. # number of input features, number of output features
        # nn.Linear bias is of shape [32] i.e. # number of output features
        
        if ((type(m) == nn.Linear) | (type(m) == nn.Conv2d)):
            torch.nn.init.xavier_uniform(m.weight)
            m.bias.data.fill_(0.00)
                

In [8]:
def return_random_agents(num_agents):
    
    agents = []
    for _ in range(num_agents):
        
        agent = CartPoleAI()
        
        for param in agent.parameters():
            param.requires_grad = False
            
        init_weights(agent)
        agents.append(agent)
        
        
    return agents
    

In [9]:
def run_agents(agents):
    
    reward_agents = []
    env = gym.make("CartPole-v0")
    
    for agent in agents:
        agent.eval()
    
        observation = env.reset()
        
        r=0
        s=0
        
        for _ in range(250):
            
            inp = torch.tensor(observation).type('torch.FloatTensor').view(1,-1)
            output_probabilities = agent(inp).detach().numpy()[0]
            action = np.random.choice(range(game_actions), 1, p=output_probabilities).item()
            new_observation, reward, done, info = env.step(action)
            r=r+reward
            
            s=s+1
            observation = new_observation

            if(done):
                break

        reward_agents.append(r)        
        #reward_agents.append(s)
        
    
    return reward_agents

In [10]:
def return_average_score(agent, runs):
    score = 0.
    for i in range(runs):
        score += run_agents([agent])[0]
    return score/runs

In [11]:
def run_agents_n_times(agents, runs):
    avg_score = []
    for agent in agents:
        avg_score.append(return_average_score(agent,runs))
    return avg_score

In [12]:
def mutate(agent):

    child_agent = copy.deepcopy(agent)
    
    mutation_power = 0.02 #hyper-parameter, set from https://arxiv.org/pdf/1712.06567.pdf
            
    for param in child_agent.parameters():
    
        if(len(param.shape)==4): #weights of Conv2D

            for i0 in range(param.shape[0]):
                for i1 in range(param.shape[1]):
                    for i2 in range(param.shape[2]):
                        for i3 in range(param.shape[3]):
                            
                            param[i0][i1][i2][i3]+= mutation_power * np.random.randn()
                                
                                    

        elif(len(param.shape)==2): #weights of linear layer
            for i0 in range(param.shape[0]):
                for i1 in range(param.shape[1]):
                    
                    param[i0][i1]+= mutation_power * np.random.randn()
                        

        elif(len(param.shape)==1): #biases of linear layer or conv layer
            for i0 in range(param.shape[0]):
                
                param[i0]+=mutation_power * np.random.randn()

    return child_agent

In [13]:
def return_children(agents, sorted_parent_indexes, elite_index):
    
    children_agents = []
    
    #first take selected parents from sorted_parent_indexes and generate N-1 children
    for i in range(len(agents)-1):
        
        selected_agent_index = sorted_parent_indexes[np.random.randint(len(sorted_parent_indexes))]
        children_agents.append(mutate(agents[selected_agent_index]))

    #now add one elite
    elite_child = add_elite(agents, sorted_parent_indexes, elite_index)
    children_agents.append(elite_child)
    elite_index=len(children_agents)-1 #it is the last one
    
    return children_agents, elite_index

In [14]:
def add_elite(agents, sorted_parent_indexes, elite_index=None, only_consider_top_n=10):
    
    candidate_elite_index = sorted_parent_indexes[:only_consider_top_n]
    
    if(elite_index is not None):
        candidate_elite_index = np.append(candidate_elite_index,[elite_index])
        
    top_score = None
    top_elite_index = None
    
    for i in candidate_elite_index:
        score = return_average_score(agents[i],runs=5)
        print("Score for elite i ", i, " is ", score)
        
        if(top_score is None):
            top_score = score
            top_elite_index = i
        elif(score > top_score):
            top_score = score
            top_elite_index = i
            
    print("Elite selected with index ",top_elite_index, " and score", top_score)
    
    child_agent = copy.deepcopy(agents[top_elite_index])
    return child_agent
    

In [15]:
def softmax(x):
    """Compute softmax values for each sets of scores in x."""
    return np.exp(x) / np.sum(np.exp(x), axis=0)

In [17]:
game_actions = 2 #2 actions possible: left or right

#disable gradients as we will not use them
torch.set_grad_enabled(False)

# initialize N number of agents
num_agents = 500
agents = return_random_agents(num_agents)

# How many top agents to consider as parents
top_limit = 20

# run evolution until X generations
generations = 1000

elite_index = None

for generation in range(generations):

    # return rewards of agents
    rewards = run_agents_n_times(agents, 3) #return average of 3 runs

    # sort by rewards
    sorted_parent_indexes = np.argsort(rewards)[::-1][:top_limit] #reverses and gives top values (argsort sorts by ascending by default) https://stackoverflow.com/questions/16486252/is-it-possible-to-use-argsort-in-descending-order
    print("")
    print("")
    
    top_rewards = []
    for best_parent in sorted_parent_indexes:
        top_rewards.append(rewards[best_parent])
    
    print("Generation ", generation, " | Mean rewards: ", np.mean(rewards), " | Mean of top 5: ",np.mean(top_rewards[:5]))
    #print(rewards)
    print("Top ",top_limit," scores", sorted_parent_indexes)
    print("Rewards for top: ",top_rewards)
    
    # setup an empty list for containing children agents
    children_agents, elite_index = return_children(agents, sorted_parent_indexes, elite_index)

    # kill all agents, and replace them with their children
    agents = children_agents



Generation  0  | Mean rewards:  21.627333333333336  | Mean of top 5:  45.266666666666666
Top  20  scores [404 491 382 456 172 236 271 321 173 415 265 376 279 178  80 445  26 467
 433 134]
Rewards for top:  [48.333333333333336, 46.666666666666664, 45.0, 43.666666666666664, 42.666666666666664, 41.666666666666664, 41.333333333333336, 41.0, 40.0, 39.0, 39.0, 39.0, 38.333333333333336, 38.333333333333336, 38.0, 37.333333333333336, 37.333333333333336, 37.0, 36.333333333333336, 36.333333333333336]
Score for elite i  404  is  18.0
Score for elite i  491  is  25.2
Score for elite i  382  is  15.2
Score for elite i  456  is  20.4
Score for elite i  172  is  31.8
Score for elite i  236  is  22.8
Score for elite i  271  is  16.2
Score for elite i  321  is  19.2
Score for elite i  173  is  21.4
Score for elite i  415  is  17.0
Elite selected with index  172  and score 31.8


Generation  1  | Mean rewards:  23.249333333333333  | Mean of top 5:  47.53333333333334
Top  20  scores [376 121 259  24 451

Score for elite i  119  is  24.8
Score for elite i  499  is  49.0
Elite selected with index  499  and score 49.0


Generation  9  | Mean rewards:  31.91133333333333  | Mean of top 5:  65.86666666666666
Top  20  scores [127 439 263 225 436 398 218 145  80 327 282 449 357  94 442 479 494 109
 394 437]
Rewards for top:  [75.0, 66.33333333333333, 66.0, 62.333333333333336, 59.666666666666664, 58.666666666666664, 58.666666666666664, 58.333333333333336, 58.333333333333336, 58.0, 57.333333333333336, 57.333333333333336, 57.333333333333336, 56.666666666666664, 56.666666666666664, 56.333333333333336, 56.333333333333336, 56.0, 55.333333333333336, 55.333333333333336]
Score for elite i  127  is  25.4
Score for elite i  439  is  29.4
Score for elite i  263  is  28.4
Score for elite i  225  is  29.4
Score for elite i  436  is  23.0
Score for elite i  398  is  29.0
Score for elite i  218  is  46.4
Score for elite i  145  is  19.6
Score for elite i  80  is  33.2
Score for elite i  327  is  48.6
Score fo

Score for elite i  83  is  20.2
Score for elite i  499  is  56.8
Elite selected with index  405  and score 58.6


Generation  18  | Mean rewards:  44.675333333333334  | Mean of top 5:  93.26666666666668
Top  20  scores [111 189  37  70 300 367 220 331  29  91 188 365 323 342 235 144  49 483
 283 481]
Rewards for top:  [101.33333333333333, 92.0, 91.66666666666667, 90.66666666666667, 90.66666666666667, 90.0, 87.0, 86.66666666666667, 86.33333333333333, 85.66666666666667, 84.66666666666667, 82.0, 81.66666666666667, 81.0, 79.33333333333333, 79.33333333333333, 78.33333333333333, 77.0, 76.66666666666667, 76.33333333333333]
Score for elite i  111  is  50.4
Score for elite i  189  is  45.8
Score for elite i  37  is  65.0
Score for elite i  70  is  29.8
Score for elite i  300  is  52.8
Score for elite i  367  is  35.0
Score for elite i  220  is  38.8
Score for elite i  331  is  48.8
Score for elite i  29  is  42.8
Score for elite i  91  is  40.6
Score for elite i  499  is  34.6
Elite selected wi



Generation  27  | Mean rewards:  51.16333333333333  | Mean of top 5:  112.66666666666667
Top  20  scores [285 152 103 390 139 483 269 477 365 267 247  88 173 147 441 128 275 199
  18 291]
Rewards for top:  [119.33333333333333, 117.0, 117.0, 105.66666666666667, 104.33333333333333, 102.0, 95.66666666666667, 95.66666666666667, 94.66666666666667, 94.33333333333333, 94.33333333333333, 92.66666666666667, 91.66666666666667, 91.66666666666667, 91.33333333333333, 91.33333333333333, 91.0, 91.0, 90.33333333333333, 90.33333333333333]
Score for elite i  285  is  44.8
Score for elite i  152  is  55.8
Score for elite i  103  is  56.2
Score for elite i  390  is  68.8
Score for elite i  139  is  62.0
Score for elite i  483  is  69.6
Score for elite i  269  is  99.6
Score for elite i  477  is  57.6
Score for elite i  365  is  58.2
Score for elite i  267  is  46.8
Score for elite i  499  is  68.2
Elite selected with index  269  and score 99.6


Generation  28  | Mean rewards:  57.14933333333334  | Mean

Score for elite i  253  is  75.6
Score for elite i  499  is  47.8
Elite selected with index  289  and score 83.6


KeyboardInterrupt: 

In [75]:
def play_agent(agent):
    try: #try and exception block because, render hangs if an erorr occurs, we must do env.close to continue working    
        env = gym.make("CartPole-v0")
        
        env_record = Monitor(env, './video', force=True)
        observation = env_record.reset()
        last_observation = observation
        r=0
        for _ in range(250):
            env_record.render()
            inp = torch.tensor(observation).type('torch.FloatTensor').view(1,-1)
            output_probabilities = agent(inp).detach().numpy()[0]
            action = np.random.choice(range(game_actions), 1, p=output_probabilities).item()
            new_observation, reward, done, info = env_record.step(action)
            r=r+reward
            observation = new_observation

            if(done):
                break

        env_record.close()
        print("Rewards: ",r)

    except Exception as e:
        env_record.close()
        print(e.__doc__)
        print(e.message)        

In [77]:
play_agent(agents[96])

Rewards:  70.0
